TBATS: Trigonometric seasonality with Box–Cox transformation, ARMA errors and optional (damped) trend. 

In [4]:
# sample usage
import numpy as np

from aeon.forecasting.machine_learning._tbats import TBATSForecaster

# Synthetic series with two seasonalities and a gentle trend
rng = np.random.default_rng(2342)
t = np.arange(160, dtype=float)
y = (
    5 * np.sin(2 * np.pi * t / 14)  # s=14
    + 2 * np.cos(2 * np.pi * t / 30.5)  # s≈30.5
    + (t / 20.0) ** 1.5  # trend
    + rng.normal(scale=t / 50.0, size=t.size)  # heteroskedastic noise
    + 10.0
)

# Create the estimator (auto-select harmonics; grid ARMA selection)
f = TBATSForecaster(
    horizon=14,
    seasonal_periods=[14, 30.5],
    use_box_cox=True,  # will be compared to no-BC and chosen by AIC
    use_trend=None,  # try with/without trend
    use_damped_trend=None,  # try damped/undamped when trend is on
    use_arma_errors=True,
    arma_selection="grid",  # or "auto_arima" (requires pmdarima)
    max_arma_order=3,
    show_warnings=True,
)

# Fit model
f.fit(y)

# Forecast 14 steps ahead (point forecasts)
y_forecast = f.fitted_forecast(steps=14)

# Forecast intervals (80% and 95%), with Box–Cox bias adjustment if applicable
mean, lower, upper, levels = f.predict_interval(steps=14, levels=(80, 95), biasadj=True)


# Summarize a few fitted artifacts
print(f"AIC: {f.aic_:.2f}")
print(
    "Chosen harmonics per season:",
    getattr(f.params_.components, "seasonal_harmonics", None),
)
print("ARMA orders (p, q):", (f.params_.components.p, f.params_.components.q))
print("First 5 forecasts:", np.round(mean[:5], 3))
# print("95% /interval for first step:", lower[0, 1], upper[0, 1])

print("levels:", levels)
print("lower.shape:", np.shape(lower), "upper.shape:", np.shape(upper))

# pick the 95% column if present, otherwise use the only column
if np.ndim(lower) == 1:
    # only one level returned
    lo95 = lower[0]
    up95 = upper[0]
else:
    # find the column for 95 (or nearest)
    i95 = int(np.argmin(np.abs(np.asarray(levels, float) - 95.0)))
    lo95 = lower[0, i95]
    up95 = upper[0, i95]

print("95% interval for first step:", lo95, up95)

AIC: 1027.65
Chosen harmonics per season: [1 5]
ARMA orders (p, q): (3, 1)
First 5 forecasts: [32.943 29.258 25.752 23.602 23.323]
levels: [80. 95.]
95% interval for first step: 30.372051438589196 35.53978281379878
